[ RAG 구현 절차 ]

```
1.	문서의 내용을 읽는다(document_loader를 이용)
(1)	https://python.langchain.com/v0.2/docs/integrations/document_loaders/ 
(2)	https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_word/
%pip install --upgrade --quiet  docx2txt
2.	문서를 쪼갠다(한번에 이해하고 처리할 수 있는 입력+출력 토큰수가 제한)
(1)	 https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/#splitting-text-from-languages-without-word-boundaries 
%pip install -qU langchain-text-splitters
3.	쪼갠 문서를 임베딩하여 vector database에 넣음
(1)	OpenAIEmbeddings나 UpstageEmbeddings이용해서 임베딩
(2)	https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/  
%pip install –q langchain-chroma
4.	질문을 이용해 유사도 검색
5.	유사도 검색한 문서를 LLM에 질문으로 전달하여 답변 얻음(제공되는 Prompt활용)
(1)	https://python.langchain.com/v0.2/docs/tutorials/rag/
%pip install –q langchain langchainhub
http://smith.langchain.com에서 key생성 .env key(LANGCHAIN_API_KEY) 추가

```

# 1. 문서 읽기(X)

In [5]:
%%time
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
document = loader.load()

CPU times: total: 4.11 s
Wall time: 4.32 s


# 2. 문서를 쪼개면서 읽기(o)

In [1]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = Docx2txtLoader('./tax_docs/소득세법(법률)(제20615호)(20250701).docx')
text_splitter = RecursiveCharacterTextSplitter(  # 문서를 쪼개는 기준이 문자수
    chunk_size=1500, #문서를 쪼갤때 1500글자씩 쪼개
    chunk_overlap=200
)
# 1번째 chunk 1~1450글자
# 2번째 chunk 1250~1750글자
document = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print('문서 쪼개면서 읽는 시간 :', runtime)

문서 쪼개면서 읽는 시간 : 3.9905669689178467


# 3. 쪼갠문서를 임베딩 -> 벡터 데이터베이스 저장
- 임베딩 모델 : upstage API의 text-embedding-3-large (기본:text-embedding-ada-002)
- 벡터 데이터베이스 : chroma

In [2]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
load_dotenv()
embedding = UpstageEmbeddings(model="solar-embedding-1-large")

In [3]:
embeddings = embedding.embed_documents(
    [
        "소득세법 어쩌구 저쩌구",
        document[0].page_content
    ]
)
len(embeddings), len(embeddings[0])

(2, 4096)

In [14]:
len(embeddings), len(embeddings[0]), len(embeddings[1])

(2, 3072, 3072)

In [15]:
len(embedding.embed_query("소득세"))

3072

In [4]:
%%time
from langchain_chroma import Chroma
#데이터를 처음 저장할 때
database = Chroma.from_documents(                                 
    documents=document,
    embedding=embedding,
    collection_name="tax-collection", # 생략시 이름 랜덤
    persist_directory='./chroma_upstage'      # 생략시 로컬데이터베이스에 저장안됨. 프로그램 종료시 db날라감
)


CPU times: total: 8.86 s
Wall time: 42.3 s


In [ ]:
# 이미 저장된 vector DB를 사용할 때
database = Chroma(
    embedding_function=embedding,
    collection_name="tax-collection",
    persist_directory='./chroma_upstage'
)

# 4. vector DB에 질문과 유사도 검색(답변 생성을 위한 retrieval)

In [5]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query,
                                           k=3) # 기본 k는 4

In [6]:
retrieved_docs

[Document(id='76791bbf-4873-44db-a827-ea9e61704d7c', metadata={'source': './tax_docs/소득세법(법률)(제20615호)(20250701).docx'}, page_content='2. 2명인 경우: 연 55만원\n\n3. 3명 이상인 경우: 연 55만원과 2명을 초과하는 1명당 연 40만원을 합한 금액\n\n② 삭제<2017. 12. 19.>\n\n③ 해당 과세기간에 출산하거나 입양 신고한 공제대상자녀가 있는 경우 다음 각 호의 구분에 따른 금액을 종합소득산출세액에서 공제한다.<신설 2015. 5. 13., 2016. 12. 20.>\n\n1. 출산하거나 입양 신고한 공제대상자녀가 첫째인 경우: 연 30만원\n\n2. 출산하거나 입양 신고한 공제대상자녀가 둘째인 경우: 연 50만원\n\n3. 출산하거나 입양 신고한 공제대상자녀가 셋째 이상인 경우: 연 70만원\n\n④ 제1항 및 제3항에 따른 공제를 “자녀세액공제”라 한다.<신설 2015. 5. 13., 2017. 12. 19.>\n\n[본조신설 2014. 1. 1.]\n\n[종전 제59조의2는 제59조의5로 이동 <2014. 1. 1.>]\n\n\n\n제59조의3(연금계좌세액공제) ① 종합소득이 있는 거주자가 연금계좌에 납입한 금액 중 다음 각 호에 해당하는 금액을 제외한 금액(이하 “연금계좌 납입액”이라 한다)의 100분의 12[해당 과세기간에 종합소득과세표준을 계산할 때 합산하는 종합소득금액이 4천 500만원 이하(근로소득만 있는 경우에는 총급여액 5천 500만원 이하)인 거주자에 대해서는 100분의 15]에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서 공제한다. 다만, 연금계좌 중 연금저축계좌에 납입한 금액이 연 600만원을 초과하는 경우에는 그 초과하는 금액은 없는 것으로 하고, 연금저축계좌에 납입한 금액 중 600만원 이내의 금액과 퇴직연금계좌에 납입한 금액을 합한 금액이 연 900만원을 초과하는 경우에는 그 초과하는 금액은 없는

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM 전달하여 답변 생성

In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")

In [8]:
prompt = f"""[identity]
- 당신은 최고의 한국 소득세 전문자입니다
- [context]를 참고해서 사용자의 질문에 답변해 주세요
[context]는 다음과 같아요
{retrieved_docs}
Question : {query}"""

In [9]:
ai_message = llm.invoke(prompt)

In [10]:
print(ai_message.content)

연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 먼저 근로소득공제, 자녀세액공제, 근로소득세액공제 등을 고려해야 합니다. 아래는 일반적인 계산 절차와 예상 세액입니다.

1. 총급여액: 50,000,000원

2. 근로소득공제
- 최대 2,000만원 공제 가능
- 연봉 5천만원은 그 이하이므로, 근로소득공제액은 2,000만원
- 과세표준 = 연봉 - 근로소득공제 = 50,000,000 - 20,000,000 = 30,000,000원

3. 과세표준이 3천만원 이하이므로, 세율은 6%입니다(그 이하 구간 기준).
- 세액 = 30,000,000 * 6% = 1,800,000원

4. 근로소득세액공제
- 종합소득산출세액 1,800,000원에 대해 근로소득세액공제(66만원) 적용
- 세액이 66만원보다 크므로, 공제액은 66만원
- 차감 후 세액 = 1,800,000 - 660,000 = 1,140,000원

5. 자녀세액공제
- 자녀수에 따라 다르나, 일반적인 경우 자녀가 없다면 공제 없음
- 자녀가 있으면 공제액이 적용되나, 질문에는 자녀 수가 언급되지 않았으므로 제외

최종 예상 소득세액은 약 **1,140,000원**입니다.

※ 참고: 실제 세액은 기타 소득공제(보험료, 의료비, 기부금 등), 지역자치세(지방세) 등을 반영하지 않은 단순 계산입니다. 구체적 세액 계산을 위해서는 상세 소득공제 내역과 세법상 공제조건을 재확인하는 것이 필요합니다.

궁금하신 점이나 구체적 소득공제 내용에 따른 맞춤 계산이 필요하시면 언제든 말씀해 주세요.


# 5. Augmentation을 위한 제공되는 Prompt 활용하여 langchain으로 답변 생성

In [11]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'

from langchain import hub

prompt = hub.pull('rlm/rag-prompt')
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

### RetrivalQA를 통해 LLM전달 (create_retrival_chain)이 대체
```
    query -> retriever전달 (벡터 검색 수행) -> retriever문서 -> prompt에 {context}에 삽입 -> query -> prompt의 {question}에 삽입 
```

In [13]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = database.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": prompt},
)

In [14]:
message = qa_chain.invoke({"query":query})
print(message)

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?', 'result': '연봉 5천만원인 직장인의 소득세는 정확하게 계산하기 어렵지만, 기본 공제와 자녀세액공제 등을 고려하면 대략 500만원에서 700만원 정도일 수 있습니다. 이 금액은 근로소득공제, 자녀세액공제, 인적공제, 연금계좌공제 등 다양한 공제 항목에 따라 달라집니다. 상세한 세액 계산은 개인의 구체적인 소득 구조와 공제 항목에 따라 차이가 있으니 세무 전문가와 상담하는 것이 좋습니다.'}
